# Deploy model to prod environment

## dev account

Login AWS console:

* aws login account: mlops-dev
* user: mlops-dev-developers

Open the cloud9 - my-mlops-works

## Deploy model to prod script

* mkdir deploy-model-to-prod
* aws --profile mlops-devops-developers codecommit create-repository --repository-name deploy-model-to-prod
* cd deploy-model-to-prod
* git init
* git remote add origin https://git-codecommit.us-west-2.amazonaws.com/v1/repos/deploy-model-to-prod
* git remote -v
* touch deploy-model-to-prod.py

Paste the following codes and commit the code into codecommit

## deploy-model-to-prod.py

In [ ]:
import boto3, sys
from urllib.parse import urlparse

prod_access_key = sys.argv[1]
prod_secrete_key = sys.argv[2]
model_name = sys.argv[3]
prod_md_bucket = sys.argv[4]

# devops sagemaker
sm_devops = boto3.client('sagemaker')
s3_devops = boto3.client('s3')
resp = sm_devops.describe_model(ModelName=model_name)

# prod sagemaker
ss_prod = boto3.Session(aws_access_key_id=prod_access_key, aws_secret_access_key=prod_secrete_key)
sm_prod = ss_prod.client('sagemaker')
s3_prod = ss_prod.client('s3')
account_prod = ss_prod.client('sts').get_caller_identity()['Account']

# copy model to prod s3
md_url_devops = resp['PrimaryContainer']['ModelDataUrl']
output = urlparse(md_url_devops)
devops_md_bucket = output.netloc
devops_md_key = output.path.lstrip('/')
source_response = s3_devops.get_object(Bucket=devops_md_bucket, Key=devops_md_key)
s3_prod.upload_fileobj(source_response['Body'],prod_md_bucket,devops_md_key)

# config prod model parameters
image = resp['PrimaryContainer']['Image']
modelDataUrl = 's3://{}/{}'.format(prod_md_bucket,devops_md_key)
model_arn = resp['ModelArn']
role = 'arn:aws:iam::{}:role/SageMakerExecutionRole'.format(account_prod)

# create prod model
create_model_response = sm_prod.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        'Image': image,
        'ModelDataUrl': modelDataUrl},
    Tags=[{"Key": "model_arn","Value": model_arn}])

print(create_model_response['ModelArn'])

# Config prod account

Login AWS console:

* aws login account: mlops-prod
* user: mlops-prod-admin

## Create S3 bucket

* bucket name: `prod-sagemaker-repo-us-west-2`

## Configure SageMaker Execution Role

Add the following policy: `prod-sagemaker-policy`

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "ecr:GetDownloadUrlForLayer",
                "ecr:BatchGetImage",
                "ecr:BatchCheckLayerAvailability"
            ],
            "Resource": [
                "arn:aws:ecr:*:<devops-account-id>:repository/*",
                "arn:aws:s3:::beta-sagemaker-repo*/*"
            ]
        }
    ]
}

# DevOps account

Login AWS console:

* aws login account: mlops-devops
* user:  mlops-devops-admin

## Secret Manger

* name: prod/sagemaker
* description: production sagemaker account
* keys: prod-sagemaker-access and prod-sagemaker-secret

## S3 Bucket Policy

Add bucket policy to devops-sagemaker-repo-us-west-2

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "AWS": "arn:aws:iam::<mlops-prod-accountid>:role/SageMakerExecutionRole"
            },
            "Action": "s3:GetObject",
            "Resource": "arn:aws:s3:::devops-sagemaker-repo-us-west-2/*"
        }
    ]
}

## Codebuild

* project name: `prod-deploy-model`
* description: `deploy model into production`
* repo: `deploy-model-to-prod`
* image: amazonlinux2
* role: `codebuild-prod-deploy-model-service-role`

In [ ]:
version: 0.2

env:
  variables:
     model_name: "<model_name>"
     prod_md_bucket: "prod-sagemaker-repo-us-west-2"
  secrets-manager:
     prod_access_key: prod/sagemaker:prod-sagemaker-access
     prod_secret_key: prod/sagemaker:prod-sagemaker-secret
phases:
  install:
    runtime-versions:
      python: 3.8
  build:
    commands:
      - python deploy-model-to-prod.py $prod_access_key $prod_secret_key $model_name $prod_md_bucket       

## Add IAM Policy to service role

* role: codebuild-prod-deploy-model-service-role
* inline policy name: `codebuild-prod-deploy-model`

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "secretsmanager:GetSecretValue",
                "sagemaker:DescribeModel"
            ],
            "Resource": [
                "arn:aws:secretsmanager:*:*:secret:prod/sagemaker-*",
                "arn:aws:sagemaker:*:*:model/*",
                "arn:aws:s3:::devops-sagemaker-repo*/*"
            ]
        }
    ]
}

## Start Build

add environment variables

* model_name and value
* prod_md_bucket

# Congraduation

You deploy the model to prod

![](./img/16.png)